In [1]:
import numpy as np
import re
import pandas as pd
import altair as alt

In [2]:
path = 'assets/'

land_df = pd.read_csv(path + 'combined_tables.csv')
chem_df = pd.read_csv(path + 'ChemDataforJeffOlson.csv')

# Format date field, drop old version

chem_df['Date']= pd.to_datetime(chem_df['VisitDate'])
chem_df.drop('VisitDate', axis=1, inplace=True)

chem_df.sample(7)

,LakeID,LakeStationNo,LakeStationType,Lat,Long,Town,ProjectID,VisitNumber,StartTime,CollectionMethodID,Depth,ActivityCategory,CharacteristicID,Symbol,Result,Calcs,ProjRemark,RemarkCode,DepthStratumCode,Date
186524,PARKER,1,Pelagic,44.71926,-72.23425,Glover,SpringTP,1,1354.0,Hydrolab,10.00,Reg,DO,NaN,9.14,Y,NaN,NaN,NaN,2000-05-25
179789,NICHOLS,1,Pelagic,44.45670,-72.33830,Woodbury,LaymonQC,1,1313.0,Hydrolab,13.02,Reg,TempC,NaN,5.94,Y,NaN,NaN,NaN,2021-07-23
25026,BRANCH,1,Pelagic,43.08079,-73.01964,Sunderland,AcidLake,1,1240.0,PlasticKemm,1.00,Reg,DCl,NaN,0.20,Y,NaN,NaN,E,2010-08-04
214520,SHADOW (CONCRD),1,Pelagic,44.40080,-71.87280,Concord,SpringTP,1,1355.0,Hydrolab,4.00,Reg,ChlaProbe,NaN,4.80,Y,NaN,NaN,NaN,2006-04-24
252400,TICKLENAKED,1,Pelagic,44.19067,-72.09890,Ryegate,LakeAsmt,1,1229.0,Hydrolab,5.00,Reg,TempC,NaN,7.81,Y,NaN,NaN,NaN,1999-05-19
44255,CASPIAN,1,Pelagic,44.58897,-72.30878,Greensboro,LakeAsmt,1,1407.0,Hydrolab,20.02,Reg,DO%,NaN,72.10,Y,NaN,NaN,NaN,2019-07-29
60865,DANBY,1,Pelagic,43.36560,-73.05220,Danby,SpringTP,1,1330.0,Hydrolab,0.20,Reg,Cond,NaN,225.00,Y,NaN,NaN,NaN,2011-04-28


In [3]:
land_df.head(10)

,Unnamed: 0,LakeID,Description,Lat,Long,Town,Shape_Length,Shape_Area,TREE_CANOPY_acres,GRASS_SHRUBS_acres,...,Shrub_Shrubs_acres,Shrub_Total_acres,TC_Coniferous_acres,TC_Deciduous_acres,TC_Total_acres,Wet_Emergent_acres,Wet_Forested_acres,Wet_Scrub_Shrub_acres,Wet_Total_acres,from_file
0,0,ABENAKI,Watershed,43.83030,-72.23610,Thetford,9683.033459,2.335672e+06,532.331074,39.856060,...,0.000000,0.000000,266.678081,265.987483,532.665564,5.913928,29.521352,0.191924,35.627204,AOIs_ABENAKI.xls
1,1,ABENAKI,Flowline100ft,43.83030,-72.23610,Thetford,6359.510779,1.864531e+05,41.212481,4.014035,...,0.000000,0.000000,19.499594,21.754112,41.253706,3.738571,3.145864,0.000000,6.884435,AOIs_ABENAKI.xls
2,2,ABENAKI,Waterbody100ft,43.83030,-72.23610,Thetford,5631.483675,8.449326e+04,17.348810,2.912812,...,0.000000,0.000000,9.122220,8.232879,17.355099,2.099783,2.956447,0.182035,5.238265,AOIs_ABENAKI.xls
3,3,ABENAKI,Buffer100ftWBFL,43.83030,-72.23610,Thetford,11439.865203,2.655565e+05,58.155297,6.164034,...,0.000000,0.000000,28.361753,29.837036,58.198789,4.854378,5.758748,0.182035,10.795161,AOIs_ABENAKI.xls
4,4,ABENAKI,Buffer250ftWaterbody,43.83030,-72.23610,Thetford,5838.954038,2.125897e+05,44.545557,5.968327,...,0.000000,0.000000,21.194604,23.420545,44.615149,3.266376,7.531832,0.191924,10.990132,AOIs_ABENAKI.xls
5,5,ADAMS (WOODFD),Watershed,42.88789,-73.03835,Woodford,11031.595410,3.073709e+06,720.717254,27.260191,...,5.385283,5.385283,120.203413,600.657779,720.861192,25.335799,90.819363,3.840259,119.995421,AOIs_ADAMSWOODFD.xls
6,6,ADAMS (WOODFD),Flowline100ft,42.88789,-73.03835,Woodford,9129.532815,2.681033e+05,46.983618,11.784561,...,1.493034,1.493034,14.957451,32.052703,47.010154,12.137647,10.578935,0.129221,22.845803,AOIs_ADAMSWOODFD.xls
7,7,ADAMS (WOODFD),Waterbody100ft,42.88789,-73.03835,Woodford,4696.987340,7.084792e+04,15.775060,0.583353,...,0.000000,0.000000,7.591926,8.177718,15.769644,0.000000,2.195926,0.000000,2.195926,AOIs_ADAMSWOODFD.xls
8,8,ADAMS (WOODFD),Buffer100ftWBFL,42.88789,-73.03835,Woodford,12999.071486,3.303234e+05,60.694610,12.322261,...,1.493034,1.493034,21.177110,39.536484,60.713594,12.137647,11.570905,0.129221,23.837773,AOIs_ADAMSWOODFD.xls
9,9,ADAMS (WOODFD),Buffer250ftWaterbody,42.88789,-73.03835,Woodford,4804.641049,1.753728e+05,40.981067,0.885501,...,0.000000,0.000000,12.842655,28.144976,40.987631,0.000000,4.522766,0.000000,4.522766,AOIs_ADAMSWOODFD.xls


For reference, these are the definitions of the geographic categories in the `Description` field at the [Vermont DNR website](https://dec.vermont.gov/watershed/lakes-ponds/data-maps/land-cover-maps):

- `Watershed`: the entire watershed.
- `Flowline100ft`: the 100-foot buffer on each side of the tributaries draining into the lake
- `Waterbody100ft`: the 100-foot buffer around the lakeshore
- `Buffer100ftWBFL`: the 100-foot buffer that runs along both sides of the tributary streams into the lake combined with the 100-foot buffer along the lakeshore
- `Buffer250ftWaterbody`: the 250-foot buffer around the lakeshore, the protected shoreland area under the Vermont Shoreland Protection Act

Note that `Shape_Area` only includes land area, not the lake itself.

## Quick tools

These functions make graphs for a fast look at the data.

`snap_chart` has arguments `lake` to identify the specific lake and `chem` for the specific chemical test.  The default data frame is `chem_df`.  It returns a scatter plot of values across the entire history of measurement.

`tre_chart` returns a scatter plot of chlorophyll-a, total phosphorus, and clairty (Secchi) for `lake`.  

In [8]:
def snap_chart(lake, chem, df=chem_df):
    ch = alt.Chart(df[ (df['LakeID'] == lake) & (df['CharacteristicID'] == chem)]).mark_circle().encode(
        alt.X('Date:T'),
        alt.Y('Result:Q')
    )
    return ch

def tre_chart(lake, df=chem_df):
    cdf = df[ df['LakeID'] == lake]
    cdf = cdf[ cdf['CharacteristicID'].apply(lambda x: x in ['TP', 'Chla', 'Secchi'])]
    ch = alt.Chart(cdf).mark_circle().encode(
        alt.X('Date:T'),
        alt.Y('Result:Q'),
        alt.Color('CharacteristicID', scale = alt.Scale( scheme = "lightmulti"))
    )
    return ch

In [9]:
snap_chart('GROUT', 'Secchi')

alt.Chart(...)

In [10]:
tre_chart('MAIDSTONE')

alt.Chart(...)

In [5]:
lakes_in_land_survey = land_df['LakeID'].unique()
all_lakes = chem_df['LakeID'].unique()

lu_df = chem_df[ chem_df['LakeID'].apply(lambda x: x in lakes_in_land_survey)].copy()

In [8]:
lake_by_meas = []
for group, frame in chem_df.groupby('LakeID'):
    lake_by_meas.append((len(frame), group))
    
sorted(lake_by_meas, reverse=True)[:10]

[(15281, 'TICKLENAKED'),
 (12896, 'CARMI'),
 (5056, 'PARKER'),
 (4406, 'MAIDSTONE'),
 (4337, 'SOUTH BAY'),
 (4139, 'BRANCH'),
 (4102, 'WILLOUGHBY'),
 (4065, 'SUNSET (MARLBR)'),
 (3944, 'GROUT'),
 (3830, 'BOURN')]

In [13]:
def most_surveyed(n, df=chem_df):
    lake_by_meas = []
    for group, frame in chem_df.groupby('LakeID'):
        lake_by_meas.append((len(frame), group))
    lake_by_meas.sort(reverse=True)
    return df[ df['LakeID'].apply(lambda x: x in [ x[1] for x in lake_by_meas ][:n])]

In [14]:
most_surveyed(10)

,LakeID,LakeStationNo,LakeStationType,Lat,Long,Town,ProjectID,VisitNumber,StartTime,CollectionMethodID,Depth,ActivityCategory,CharacteristicID,Symbol,Result,Calcs,ProjRemark,RemarkCode,DepthStratumCode,Date
18546,BOURN,1,Pelagic,43.10695,-73.00224,Sunderland,AcidLake,1,1500.0,Secchi,NaN,Reg,Secchi,NaN,2.30,Y,NaN,NaN,NaN,1982-08-17
18547,BOURN,1,Pelagic,43.10695,-73.00224,Sunderland,AcidLake,1,1500.0,TygonHose,6.00,Reg,Cond,NaN,16.00,Y,NaN,NaN,NaN,1982-08-17
18548,BOURN,1,Pelagic,43.10695,-73.00224,Sunderland,AcidLake,1,1500.0,TygonHose,6.00,Reg,GranAlk,NaN,0.22,Y,NaN,NaN,NaN,1982-08-17
18549,BOURN,1,Pelagic,43.10695,-73.00224,Sunderland,AcidLake,1,1500.0,TygonHose,6.00,Reg,pH,NaN,5.23,Y,NaN,NaN,NaN,1982-08-17
18550,BOURN,1,Pelagic,43.10695,-73.00224,Sunderland,AcidLake,1,1500.0,TygonHose,6.00,Reg,SpectroDisColor,NaN,58.00,Y,NaN,NaN,NaN,1982-08-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281474,WILLOUGHBY,1,Pelagic,44.74886,-72.06116,Westmore,LaymonQC,1,1411.0,Hydrolab,65.23,Reg,Cond,NaN,49.90,Y,NaN,NaN,NaN,2021-07-12
281475,WILLOUGHBY,1,Pelagic,44.74886,-72.06116,Westmore,LaymonQC,1,1411.0,Hydrolab,65.23,Reg,DO,NaN,11.78,Y,NaN,NaN,NaN,2021-07-12
281476,WILLOUGHBY,1,Pelagic,44.74886,-72.06116,Westmore,LaymonQC,1,1411.0,Hydrolab,65.23,Reg,DO%,NaN,90.10,Y,NaN,NaN,NaN,2021-07-12
281477,WILLOUGHBY,1,Pelagic,44.74886,-72.06116,Westmore,LaymonQC,1,1411.0,Hydrolab,65.23,Reg,pH,NaN,7.26,Y,NaN,NaN,NaN,2021-07-12


In [6]:
! dir

 Volume in drive C is Acer
 Volume Serial Number is E800-62FA

 Directory of C:\Users\algeb\OneDrive\Documents\MADS\SIADS 591\Vermont-Lake-Health

12/24/2021  05:14 PM    <DIR>          .
12/24/2021  08:18 AM    <DIR>          ..
12/24/2021  08:18 AM             1,928 .gitignore
12/24/2021  03:35 PM    <DIR>          .ipynb_checkpoints
12/24/2021  03:35 PM    <DIR>          assets
12/24/2021  08:18 AM            11,605 geomaping.ipynb
12/24/2021  08:18 AM           214,632 image.jpg
12/24/2021  08:18 AM             1,087 LICENSE
12/24/2021  08:18 AM               463 README.md
12/24/2021  03:40 PM             3,381 sshkey
12/24/2021  03:40 PM               744 sshkey.pub
12/24/2021  05:14 PM         1,021,956 VTLakes1.ipynb
               8 File(s)      1,255,796 bytes
               4 Dir(s)  384,461,922,304 bytes free


In [7]:
!git --version

git version 2.34.1.windows.1


In [8]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   VTLakes1.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   VTLakes1.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	assets/
	sshkey
	sshkey.pub

